In [1]:
import rasterio
import rasterio.plot
import numpy as np
from osgeo import gdal_array
from scipy.signal import savgol_filter
from scipy.interpolate import pchip_interpolate, InterpolatedUnivariateSpline
import math
from scipy.optimize import minimize
import ad
import matplotlib.pyplot as plt
%matplotlib inline


# 滤波

## 输入数据要求
- **时间序列影像**：`GeoTiff` 格式的时间序列影像（`*.tif`），其中每个波段表示不同日期的 NDVI 影像，第一个波段为这年第001天的影像，第二个波段为第017天的影像，以此类推。

【注意】为了减小文件大小，输入的NDVI值被乘以$10000$，然后以`Int 16`整形存储。如果需要获得真实NDVI需要乘以比例系数$0.00001$. 比例系数不会影响物候期提取，可以不乘。

## 输出数据格式
- **SG滤波后的时间序列影像**：`GeoTiff` 格式的时间序列影像（`*.tif`），格式与输入数据要求一致，输出的文件名为 `<年份>.tif`，其中`<年份>`为影像的年份，格式如：2019, 2018等等。

### 注意
1. 由于影像文件较大，滤波然后保存文件可能会比较耗时。
2. 在运行此脚本前，请清空所有不必要的内存，否则无法分配内存运算，导致程序异常退出。

In [2]:
# 时间和日期定义
years = range(2019, 2020) # 定义年份
dates = np.arange(1, 365, 16).tolist() # 定义日期，MODIS从每年第一天到365天之间每隔16天一景
dates[1:20]

for year in years:
    raw_img_path = "data/" + str(year) + '.tif'
    out_img_path = "data/" + str(year) + '_filtered.tif'
    raw_img = gdal_array.LoadFile(raw_img_path)
    src = rasterio.open("data/" + str(year) + '.tif')
    [doy, img_ht, img_wd] = raw_img.shape
    filtered_img = np.zeros((doy, img_ht, img_wd))
    
    # SG滤波，参数可调整
    for x in range(1,img_ht):
        # 打印进度信息，因为SG滤波耗时较长。
        if x%100 == 0:
            print(x, ' of ', img_ht, '\t', x/img_ht *100, '%')
        for y in range(1,img_wd):
            series = raw_img[:,x,y] # 获取原始影像时间序列
            if np.count_nonzero(series) > 20:
                filtered_series = savgol_filter(series, 5, 3) # SG滤波
                filtered_img[:,x,y] = filtered_series # 滤波后存储到新影像中
            else:
                continue
   
    # 把滤波后的时间序列存储到新的影像中
    with rasterio.open(
        out_img_path,
        'w',
        driver='GTiff',
        height=filtered_img.shape[1],
        width=filtered_img.shape[2],
        count=filtered_img.shape[0],
        dtype=rasterio.int16, #存储数据精度为 Int16
        crs=src.crs,
        transform=src.transform,
    ) as dst:
        dst.write(filtered_img.astype(np.int16)) 

    
plt.imshow(filtered_img[10,:,:])
print(filtered_img.shape)

100  of  6359 	 1.5725743041358704 %
200  of  6359 	 3.145148608271741 %
300  of  6359 	 4.717722912407611 %
400  of  6359 	 6.290297216543482 %
500  of  6359 	 7.862871520679351 %
600  of  6359 	 9.435445824815222 %
700  of  6359 	 11.008020128951093 %
800  of  6359 	 12.580594433086963 %
900  of  6359 	 14.153168737222835 %
1000  of  6359 	 15.725743041358703 %
1100  of  6359 	 17.298317345494574 %
1200  of  6359 	 18.870891649630444 %
1300  of  6359 	 20.443465953766314 %
1400  of  6359 	 22.016040257902187 %
1500  of  6359 	 23.588614562038057 %
1600  of  6359 	 25.161188866173926 %
1700  of  6359 	 26.7337631703098 %
1800  of  6359 	 28.30633747444567 %
1900  of  6359 	 29.87891177858154 %
2000  of  6359 	 31.451486082717405 %
2100  of  6359 	 33.02406038685328 %
2200  of  6359 	 34.59663469098915 %
2300  of  6359 	 36.16920899512502 %
2400  of  6359 	 37.74178329926089 %
2500  of  6359 	 39.31435760339676 %
2600  of  6359 	 40.88693190753263 %
2700  of  6359 	 42.459506211668504 

ValueError: the array's dtype 'float64' does not match the file's dtype 'int16'